In [1]:
import pandas as pd
import os
import shutil

# Ruta a la carpeta donde están tus archivos CSV
carpeta = "ClimateDataStore"

# Diccionario de mapeo para renombrar archivos
mapeo_archivos = {
     "01_mean_temperature": "Temp media",
    "03_heating_degree_days": "Dias calefaccion",
    "04_cooling_degree_days": "Dias refrigeracion",
    "11_frost_days": "Dias helada",
    "l1_daily_maximum_temperature-reanalysis-monthly-max-layer-": "Max temp monthly max",
    "l1_daily_maximum_temperature-reanalysis-monthly-mean-layer": "Max temp monthly mean",
    "l1_daily_maximum_temperature-reanalysis-monthly-min-layer": "Max temp monthly min",
    "l2_daily_minimum_temperature-reanalysis-monthly-max-layer": "Min temp monthly max",
    "l2_daily_minimum_temperature-reanalysis-monthly-mean-layer": "Min temp monthly mean",
    "l2_daily_minimum_temperature-reanalysis-monthly-min-layer": "Min temp monthly min"
}

# Lista para almacenar DataFrames
dfs = []

# Paso 1: Copiar y procesar los archivos originales
for archivo in os.listdir(carpeta):
    # Ignorar archivos que no son CSV o que sean el resultado final
    if not archivo.endswith(".csv") or archivo == "datos_climaticos_unidos.csv":
        continue
        
    ruta_completa = os.path.join(carpeta, archivo)
    
    # Determinar el nuevo nombre para la columna
    nombre_columna = None
    for clave, valor in mapeo_archivos.items():
        if clave in archivo:
            nombre_columna = valor
            break
    
    if nombre_columna is None:
        print(f"Ignorando archivo sin coincidencia: {archivo}")
        continue
    
    # Leer el archivo CSV
    try:
        df = pd.read_csv(ruta_completa)
        
        # Filtrar columnas
        cols_a_mantener = ['time']
        for col in df.columns:
            if col not in ['time', 'transnational_regions', 'realization']:
                cols_a_mantener.append(col)
        
        df_reduced = df[cols_a_mantener].copy()
        
        # Renombrar las columnas de valor (excepto time)
        for col in df_reduced.columns:
            if col != 'time':
                df_reduced = df_reduced.rename(columns={col: nombre_columna})
        
        # Convertir time a datetime si es posible
        try:
            df_reduced['time'] = pd.to_datetime(df_reduced['time'])
        except:
            pass  # Si no se puede convertir, dejarlo como está
        
        # Guardar DataFrame
        dfs.append(df_reduced)
        print(f"Procesado: {archivo} -> {nombre_columna}")
    except Exception as e:
        print(f"Error al procesar {archivo}: {e}")

# Paso 2: Unir todos los DataFrames
if dfs:
    # Establecer 'time' como índice para todos los DataFrames
    for i in range(len(dfs)):
        dfs[i] = dfs[i].set_index('time')
    
    # Unir todos los DataFrames en uno solo
    df_final = pd.concat(dfs, axis=1)
    
    # Si hay columnas con el mismo nombre, agregar sufijo
    df_final = df_final.loc[:,~df_final.columns.duplicated()]
    
    # Restablecer el índice para que 'time' vuelva a ser una columna
    df_final = df_final.reset_index()
    
    # Ordenar el DataFrame final por la columna time
    df_final = df_final.sort_values('time')
    
    # Guardar el DataFrame final como CSV
    ruta_final = os.path.join(carpeta, "datos_climaticos_unidos_limpio.csv")
    df_final.to_csv(ruta_final, index=False)
    
    print(f"Archivo final generado: datos_climaticos_unidos_limpio.csv")
    print(f"Columnas en el archivo final: {', '.join(df_final.columns)}")
else:
    print("No se encontraron archivos CSV para procesar.")

Procesado: 01_mean_temperature-reanalysis-monthly-layer-eea_trans_mediterranean-latitude-1940-2023-v1.0.csv -> Temp media
Procesado: 03_heating_degree_days-reanalysis-monthly-layer-eea_trans_mediterranean-latitude-1940-2023-v1.0.csv -> Dias calefaccion
Procesado: 04_cooling_degree_days-reanalysis-monthly-layer-eea_trans_mediterranean-latitude-1940-2023-v1.0.csv -> Dias refrigeracion
Procesado: 11_frost_days-reanalysis-monthly-layer-eea_trans_mediterranean-latitude-1940-2023-v1.0.csv -> Dias helada
Procesado: l1_daily_maximum_temperature-reanalysis-monthly-max-layer-eea_trans_mediterranean-latitude-1940-2023-v1.0.csv -> Max temp monthly max
Procesado: l1_daily_maximum_temperature-reanalysis-monthly-mean-layer-eea_trans_mediterranean-latitude-1940-2023-v1.0.csv -> Max temp monthly mean
Procesado: l1_daily_maximum_temperature-reanalysis-monthly-min-layer-eea_trans_mediterranean-latitude-1940-2023-v1.0.csv -> Max temp monthly min
Procesado: l2_daily_minimum_temperature-reanalysis-monthly-m

In [11]:
df_final.head()

,time,Temp media,Dias calefaccion,Dias efrigeracion,Dias helada,Max temp monthly max,Max temp monthly mean,Max temp monthly min,Min temp monthly max,Min temp monthly mean,Min temp monthly min
0,1940-01-01,274.075920,451.715785,0.000000,17.126696,283.069720,277.383849,269.902017,278.237537,271.128065,262.126020
1,1940-02-01,277.556366,321.482865,0.001133,11.215833,287.332116,281.234155,272.256102,279.754464,274.282298,265.125066
2,1940-03-01,280.152728,266.586311,0.185722,8.919249,292.312914,284.259968,276.465635,283.515866,276.474440,269.207108
3,1940-04-01,283.823841,156.481179,0.591104,2.986531,294.499065,287.964766,281.143995,285.060999,279.994870,274.153194
4,1940-05-01,288.069310,69.057994,5.698343,0.450916,298.847128,292.079620,285.651978,288.953641,284.150881,280.160487


In [8]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1008 entries, 0 to 1007
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   time                   1008 non-null   datetime64[ns]
 1   Temp media             1008 non-null   float64       
 2   Dias calefaccion       1008 non-null   float64       
 3   Dias efrigeracion      1008 non-null   float64       
 4   Dias helada            1008 non-null   float64       
 5   Max temp monthly max   1008 non-null   float64       
 6   Max temp monthly mean  1008 non-null   float64       
 7   Max temp monthly min   1008 non-null   float64       
 8   Min temp monthly max   1008 non-null   float64       
 9   Min temp monthly mean  1008 non-null   float64       
 10  Min temp monthly min   1008 non-null   float64       
dtypes: datetime64[ns](1), float64(10)
memory usage: 86.8 KB


In [9]:
#exportamos el archivo
df_final.to_csv('datos_climaticos_unidos_limpio.csv', index=False)